In [40]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import numpy.typing as npt
import cProfile

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


# create a seed
seed = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=np.int8)

# create a biome
biome = np.zeros((100, 100), dtype=np.int8)
biome[0:10, 0:10] = seed




In [41]:
class Biome():
    def __init__(self, grid: npt.NDArray[np.int8]):
        self.grid = grid

    def run(self, generations: int):
        for i in range(generations):
            self.grid = self._update(self.grid)
        
    def _update(self, grid: npt.NDArray[np.int8]) -> npt.NDArray[np.int8]:
        
        if np.sum(grid) == 0:
            return grid

        # get indices of all non-zero elements
        indices = np.argwhere(grid == 1)
   
        # get the surrounding indices
        surrounding_indices = np.array([[x + i, y + j] for x, y in indices for i in range(-1, 2) for j in range(-1, 2) if not (i == 0 and j == 0) ])
      
        # remove the indices that are out of bounds
        surrounding_indices = surrounding_indices[(surrounding_indices[:, 0] >= 0) 
                                                & (surrounding_indices[:, 0] < grid.shape[0])
                                                & (surrounding_indices[:, 1] >= 0) 
                                                & (surrounding_indices[:, 1] < grid.shape[1])]

        # for each surrounding index that occures three times, set the biome to 1
        # for each surrounding index that occures three times, set the biome to 1
        for i in np.unique(surrounding_indices, axis=0):
            if np.sum(np.all(surrounding_indices == i, axis=1)) == 3:
                grid[i[0], i[1]] = 1
            elif grid[i[0], i[1]] == 1 and np.sum(np.all(surrounding_indices == i, axis=1)) == 2:
                grid[i[0], i[1]] = 1
            else:
                grid[i[0], i[1]] = 0
        
        return grid
    
    def __repr__(self):
        return str(self.grid)

biome = Biome(biome)
cProfile.run('biome.run(5)')

         2768 function calls (2738 primitive calls) in 0.007 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 1008131919.py:18(<listcomp>)
        1    0.000    0.000    0.007    0.007 1008131919.py:5(run)
        5    0.002    0.000    0.007    0.001 1008131919.py:9(_update)
      127    0.000    0.000    0.001    0.000 <__array_function__ internals>:177(all)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(argwhere)
       10    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(moveaxis)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(ndim)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(nonzero)
        5    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(prod)
      132    0.000    0.000    0.002    0.000 <__array_function__ internals>:177(su